In [1]:
import findspark
findspark.init('C:\spark-3.4.1-bin-hadoop3')

import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("globant_test") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.6,com.microsoft.azure:azure-storage:8.6.6") \
    .config("spark.sql.sources.partitionOverwriteMode","dynamic")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
import configparser
config = configparser.RawConfigParser()
config.read('config.ini')

account_key = config['azure']['accountKey']
account_name = config['azure']['accountName']
raw_container = config['azure']['rawContainer']
silver_container = config['azure']['silverContainer']

spark._jsc.hadoopConfiguration().set(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key)

raw_fs = f'abfss://{raw_container}@{account_name}.dfs.core.windows.net'
hmz_fs = f'abfss://{silver_container}@{account_name}.dfs.core.windows.net'
file_path = 'globant'

In [3]:
%run -i harmonize.ipynb

In [4]:
hmz.harmonize(raw_fs, file_path)

In [ ]:
hmz.writeSilver(hmz_fs, file_path)